# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [27]:
# 学号：0234972
# 姓名：王雅婷
# 专业：计算机科学与技术

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error
import warnings

plt.rcParams["font.family"] = ["SimSun", "Microsoft YaHei", "SimHei"]  
plt.rcParams["axes.unicode_minus"] = False  
warnings.filterwarnings('ignore')

# 1. 数据导入与预处理
csv_path = "E:/Demo/python/回归分析实践—某车企汽车年销量预测/data/数据-汽车销量预测.csv" 
try:
    df_0234972 = pd.read_csv(csv_path)
    print(f"CSV数据导入成功，共{df_0234972.shape[0]}行{df_0234972.shape[1]}列")
    print("数据列名：", df_0234972.columns.tolist())  
except FileNotFoundError:
    raise FileNotFoundError(f"未找到CSV文件，请检查路径：{csv_path}")
except Exception as e:
    raise Exception(f"数据读取错误：{str(e)}")

# 数据基本信息查看（质量审核）
print("\n【数据源质量审核结果】")
print("1. 缺失值检查：")
missing_0234972 = df_0234972.isnull().sum()
print(missing_0234972[missing_0234972 > 0] if any(missing_0234972 > 0) else "无缺失值")

print("\n2. 异常值检查（数值型变量）：")
num_cols_0234972 = df_0234972.select_dtypes(include=[np.number]).columns
for col in num_cols_0234972:
    # 绘制箱线图检测异常值
    plt.figure(figsize=(6, 3))
    sns.boxplot(x=df_0234972[col])
    plt.title(f"图1. {col}的异常值检测（学号0234972）")
    plt.savefig(f"异常值检测_{col}_0234972.png")
    plt.close()
    print(f"已生成{col}的异常值检测图（图1）")

# 数据预处理（处理缺失值和异常值）
print("\n【数据预处理】")
# 填充缺失值（数值型用均值，类别型用众数）
for col in df_0234972.columns:
    if df_0234972[col].isnull().any():
        if df_0234972[col].dtype in [np.int64, np.float64]:
            df_0234972[col].fillna(df_0234972[col].mean(), inplace=True)
            print(f"数值型字段'{col}'缺失值已用均值填充")
        else:
            df_0234972[col].fillna(df_0234972[col].mode()[0], inplace=True)
            print(f"类别型字段'{col}'缺失值已用众数填充")

# 异常值处理（1.5倍四分位距截断）
for col in num_cols_0234972:
    q1 = df_0234972[col].quantile(0.25)
    q3 = df_0234972[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    df_0234972[col] = df_0234972[col].clip(lower, upper)  # 截断异常值
print("异常值已通过1.5倍四分位距法处理")


year_col = "year"       
sales_col = "sales"    

# 重命名为统一字段名（方便后续代码调用）
if year_col in df_0234972.columns:
    df_0234972 = df_0234972.rename(columns={year_col: "year"})
else:
    raise ValueError(f"未找到年份字段，请检查列名是否为'{year_col}'")

if sales_col in df_0234972.columns:
    df_0234972 = df_0234972.rename(columns={sales_col: "sales"})
else:
    raise ValueError(f"未找到销量字段，请检查列名是否为'{sales_col}'")

print(f"字段映射完成：'{year_col}'→'year'，'{sales_col}'→'sales'")
print("预处理后数据预览：")
print(df_0234972[["year", "sales"]].head())  # 预览核心字段

# --------------------------
# 2. 线性回归拟合
# --------------------------
print("\n【线性回归拟合】")
X_lin_0234972 = df_0234972[["year"]]  # 自变量：年份
y_lin_0234972 = df_0234972["sales"]   # 因变量：销量

# 建立模型
model_lin_0234972 = LinearRegression()
model_lin_0234972.fit(X_lin_0234972, y_lin_0234972)
y_pred_lin_0234972 = model_lin_0234972.predict(X_lin_0234972)

# 模型评估
r2_lin_0234972 = r2_score(y_lin_0234972, y_pred_lin_0234972)
mse_lin_0234972 = mean_squared_error(y_lin_0234972, y_pred_lin_0234972)
print(f"线性回归R²值：{r2_lin_0234972:.4f}（越接近1拟合越好）")
print(f"线性回归MSE值：{mse_lin_0234972:.4f}（值越小误差越小）")
print(f"线性回归方程：销量 = {model_lin_0234972.coef_[0]:.4f}×年份 + {model_lin_0234972.intercept_:.4f}")

# 可视化
plt.figure(figsize=(10, 6))
plt.scatter(X_lin_0234972, y_lin_0234972, color='blue', label='实际销量')
plt.plot(X_lin_0234972, y_pred_lin_0234972, color='red', label=f'线性拟合（R²={r2_lin_0234972:.4f}）')
plt.xlabel('年份')
plt.ylabel('汽车销量')
plt.title('图2. 线性回归拟合结果（学号0234972）')
plt.legend()
plt.savefig('线性回归拟合_0234972.png')
plt.close()
print("已生成线性回归拟合图（图2）")

# --------------------------
# 3. 曲线拟合（多项式回归）
# --------------------------
print("\n【曲线拟合（多项式回归）】")
best_degree_0234972 = 2
best_r2_0234972 = 0
poly_models_0234972 = {}

# 尝试2-4阶多项式，选择最优阶数
for degree in [2, 3, 4]:
    poly_0234972 = PolynomialFeatures(degree=degree)
    X_poly_0234972 = poly_0234972.fit_transform(X_lin_0234972)
    
    model_poly_0234972 = LinearRegression()
    model_poly_0234972.fit(X_poly_0234972, y_lin_0234972)
    y_pred_poly_0234972 = model_poly_0234972.predict(X_poly_0234972)
    
    r2_poly_0234972 = r2_score(y_lin_0234972, y_pred_poly_0234972)
    poly_models_0234972[degree] = (model_poly_0234972, poly_0234972, r2_poly_0234972)
    
    if r2_poly_0234972 > best_r2_0234972:
        best_r2_0234972 = r2_poly_0234972
        best_degree_0234972 = degree

print(f"最优多项式阶数：{best_degree_0234972}阶，R²值：{best_r2_0234972:.4f}")

# 可视化最优多项式
best_model_poly_0234972, best_poly_0234972, _ = poly_models_0234972[best_degree_0234972]
X_poly_best_0234972 = best_poly_0234972.fit_transform(X_lin_0234972)
y_pred_poly_best_0234972 = best_model_poly_0234972.predict(X_poly_best_0234972)

plt.figure(figsize=(10, 6))
plt.scatter(X_lin_0234972, y_lin_0234972, color='blue', label='实际销量')
plt.plot(X_lin_0234972, y_pred_poly_best_0234972, color='green', 
         label=f'{best_degree_0234972}阶多项式拟合（R²={best_r2_0234972:.4f}）')
plt.xlabel('年份')
plt.ylabel('汽车销量')
plt.title('图3. 多项式回归拟合结果（学号0234972）')
plt.legend()
plt.savefig(f'{best_degree_0234972}阶多项式拟合_0234972.png')
plt.close()
print(f"已生成{best_degree_0234972}阶多项式拟合图（图3）")

# --------------------------
# 4. 非线性回归拟合（指数模型）
# --------------------------
print("\n【非线性回归拟合（指数模型）】")
# 指数模型：y = a*e^(b*x) → 转换为ln(y) = ln(a) + b*x
y_log_0234972 = np.log(y_lin_0234972.replace(0, np.nan).dropna())  # 去除0值（避免log(0)错误）
X_log_0234972 = X_lin_0234972.loc[y_log_0234972.index]  # 对齐索引

# 建立模型
model_exp_0234972 = LinearRegression()
model_exp_0234972.fit(X_log_0234972, y_log_0234972)
y_pred_log_0234972 = model_exp_0234972.predict(X_log_0234972)
y_pred_exp_0234972 = np.exp(y_pred_log_0234972)  # 转换回原尺度

# 模型评估
r2_exp_0234972 = r2_score(y_lin_0234972.loc[y_log_0234972.index], y_pred_exp_0234972)
print(f"指数回归R²值：{r2_exp_0234972:.4f}")
print(f"指数回归方程：销量 = e^({model_exp_0234972.intercept_:.4f} + {model_exp_0234972.coef_[0]:.4f}×年份)")

# 可视化
plt.figure(figsize=(10, 6))
plt.scatter(X_lin_0234972, y_lin_0234972, color='blue', label='实际销量')
plt.plot(X_log_0234972, y_pred_exp_0234972, color='purple', 
         label=f'指数拟合（R²={r2_exp_0234972:.4f}）')
plt.xlabel('年份')
plt.ylabel('汽车销量')
plt.title('图4. 指数回归拟合结果（学号0234972）')
plt.legend()
plt.savefig('指数回归拟合_0234972.png')
plt.close()
print("已生成指数回归拟合图（图4）")

# --------------------------
# 5. 模型比较与未来销量预测
# --------------------------
print("\n【模型比较与未来销量预测】")
# 模型性能汇总
models_compare_0234972 = pd.DataFrame({
    '模型类型': ['线性回归', f'{best_degree_0234972}阶多项式回归', '指数回归'],
    'R²值': [r2_lin_0234972, best_r2_0234972, r2_exp_0234972]
})
print("模型性能比较：")
print(models_compare_0234972)

# 选择最优模型（R²最大）
best_model_type_0234972 = models_compare_0234972.loc[models_compare_0234972['R²值'].idxmax()]['模型类型']
print(f"\n最优模型：{best_model_type_0234972}（R²值最高，拟合效果最好）")

# 预测未来2-3年销量
last_year_0234972 = df_0234972['year'].max()
future_years_0234972 = pd.DataFrame({
    'year': [last_year_0234972 + 1, last_year_0234972 + 2, last_year_0234972 + 3]
})

# 根据最优模型预测
if best_model_type_0234972 == '线性回归':
    future_pred_0234972 = model_lin_0234972.predict(future_years_0234972)
elif '多项式' in best_model_type_0234972:
    future_pred_0234972 = best_model_poly_0234972.predict(
        best_poly_0234972.transform(future_years_0234972)
    )
else:  # 指数回归
    future_pred_log_0234972 = model_exp_0234972.predict(future_years_0234972)
    future_pred_0234972 = np.exp(future_pred_log_0234972)

# 整理预测结果
pred_results_0234972 = pd.DataFrame({
    '预测年份': future_years_0234972['year'],
    '预测销量': future_pred_0234972.round(2)
})
print("\n未来3年销量预测结果：")
print(pred_results_0234972)

# 可视化预测结果
plt.figure(figsize=(10, 6))
plt.scatter(df_0234972['year'], df_0234972['sales'], color='blue', label='历史销量')
plt.scatter(future_years_0234972['year'], future_pred_0234972, color='red', marker='*', s=100, label='预测销量')

# 绘制最优模型拟合线
if best_model_type_0234972 == '线性回归':
    plt.plot(df_0234972['year'], y_pred_lin_0234972, color='red', label='拟合线')
elif '多项式' in best_model_type_0234972:
    plt.plot(df_0234972['year'], y_pred_poly_best_0234972, color='green', label='拟合线')
else:
    plt.plot(X_log_0234972, y_pred_exp_0234972, color='purple', label='拟合线')

plt.xlabel('年份')
plt.ylabel('汽车销量')
plt.title(f'图5. 最优模型预测结果（学号0234972，模型：{best_model_type_0234972}）')
plt.legend()
plt.savefig('销量预测结果_0234972.png')
plt.close()
print("已生成销量预测结果图（图5）")

# 保存预测结果为CSV
pred_results_0234972.to_csv('销量预测结果_0234972.csv', index=False)
print("\n程序执行完成！输出文件清单：")
print("- 异常值检测图（多个PNG文件）")
print("- 线性回归拟合图（线性回归拟合_0234972.png）")
print("- 多项式拟合图（如2阶多项式拟合_0234972.png）")
print("- 指数回归拟合图（指数回归拟合_0234972.png）")
print("- 销量预测结果图（销量预测结果_0234972.png）")
print("- 预测结果数据（销量预测结果_0234972.csv）")



CSV数据导入成功，共14行2列
数据列名： ['year', 'sales']

【数据源质量审核结果】
1. 缺失值检查：
无缺失值

2. 异常值检查（数值型变量）：
已生成year的异常值检测图（图1）
已生成sales的异常值检测图（图1）

【数据预处理】
异常值已通过1.5倍四分位距法处理
字段映射完成：'year'→'year'，'sales'→'sales'
预处理后数据预览：
     year  sales
0  1988.0   65.0
1  1989.0   59.0
2  1990.0   51.0
3  1991.0   71.0
4  1992.0  106.0

【线性回归拟合】
线性回归R²值：0.9430（越接近1拟合越好）
线性回归MSE值：169.9155（值越小误差越小）
线性回归方程：销量 = 13.1560×年份 + -26107.4440
已生成线性回归拟合图（图2）

【曲线拟合（多项式回归）】
最优多项式阶数：4阶，R²值：0.9476
已生成4阶多项式拟合图（图3）

【非线性回归拟合（指数模型）】
指数回归R²值：0.9231
指数回归方程：销量 = e^(-215.2367 + 0.1103×年份)
已生成指数回归拟合图（图4）

【模型比较与未来销量预测】
模型性能比较：
      模型类型       R²值
0     线性回归  0.943029
1  4阶多项式回归  0.947569
2     指数回归  0.923067

最优模型：4阶多项式回归（R²值最高，拟合效果最好）

未来3年销量预测结果：
     预测年份    预测销量
0  2002.0  248.04
1  2003.0  271.57
2  2004.0  297.72
已生成销量预测结果图（图5）

程序执行完成！输出文件清单：
- 异常值检测图（多个PNG文件）
- 线性回归拟合图（线性回归拟合_0234972.png）
- 多项式拟合图（如2阶多项式拟合_0234972.png）
- 指数回归拟合图（指数回归拟合_0234972.png）
- 销量预测结果图（销量预测结果_0234972.png）
- 预测结果数据（销量预测结果_0234972.csv）


In [ ]:
df_0234972 = pd.read_csv("数据-汽车销量预测.csv")
